In [ ]:
#https://www.kaggle.com/huanghaicheng1024/gan-mnist
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model, load_model
from keras.optimizers import Adam

%matplotlib inline
import matplotlib.pyplot as plt

import sys

import numpy as np

4


In [ ]:
#新建資料夾

def mkdir(path):
  #引入模組
  import os

  #去除空格
  path = path.strip()
  #去除尾部\符號
  path = path.rstrip("\\")

  #判斷路徑是否存在
  #存在   True
  #不存在  False
  isExists = os.path.exists(path)

  #判斷結果
  if not isExists:
    #如果不存在創建目錄
    print (path + ' 創建成功')
    #創建目錄函數
    os.makedirs(path)
    return True
  else:
    #如果目錄存在不創建，提示目錄已存在
    print (path + ' 目錄存在')
    return False

#定義要創建的目錄
mkpath = "./pic3"
#調用函數
mkdir(mkpath)

In [ ]:
#模型
class GAN():
    def __init__(self):
        #各類參數
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 128  #

        optimizer = Adam(0.0002,0.5)  #Adam優化器

        #建構神經網路
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])  #準確度

        #建構生成器
        self.generator = self.build_generator()

        #生成圖片
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        #組合模型訓練
        self.discriminator.trainable = False

        #驗證
        validity = self.discriminator(img)

        #組合模型
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

In [ ]:
def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh')) 
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

In [ ]:
def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))  #二維圖片
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

In [ ]:
def load_data(self, path):
        data = np.load(path,allow_pickle=True)
        x_train = data['x_train']
        y_train = data['y_train']
        x_test = data['x_test']
        y_test = data['y_test']
        data.close()
        return (x_train, y_train), (x_test, y_test)

In [ ]:
def train(self, epochs, batch_size=128, sample_interval=50, print_log=True):

        #載入數據
        (X_train, _), (X_test, _) = self.load_data('/kaggle/input/mnist-numpy/mnist.npz')

        X_train = np.concatenate([X_train, X_test], axis=0)
        
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)  #加入一维channels

        #混淆矩陣
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        #保存loss
        all_loss = np.zeros((epochs,3))
        
        for epoch in range(epochs):

            #---------------------
            #訓練
            #---------------------

            #隨機抽取圖片batch
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            #生成batch標準常態分佈
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            #生成圖片
            gen_imgs = self.generator.predict(noise)

            #訓練
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)  # 希望对真实图片辨别为1
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)  # 希望对生成图片辨别为0
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)  # 总loss

            #---------------------
            #訓練
            #---------------------

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            #訓練
            g_loss = self.combined.train_on_batch(noise, valid)  # 希望对输入噪声生成图片辨别为1

            #輸出
            if print_log:
                print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            #保存loss
            all_loss[epoch,0] = d_loss[0]
            all_loss[epoch,1] = d_loss[1]
            all_loss[epoch,2] = g_loss
            
            #每一段epoch保存圖片
            if epoch<=1000:
                if epoch % 100 == 0:
                    self.sample_images(epoch)
                
        #繪製loss圖表
        self.plot_loss(all_loss)

In [ ]:
def sample_images(self, epoch, figsize=5):
        #生成圖片

        n = 5  #n*n個圖片
        figure = np.zeros((self.img_rows * n, self.img_cols * n))  #預先定義圖片矩陣
        noise = np.random.normal(0, 1, (n * n, self.latent_dim))
        gen_imgs = self.generator.predict(noise)
        gen_imgs = 0.5 * gen_imgs + 0.5
        for i in range(n):
            for j in range(n):
                figure[i * self.img_rows: (i + 1) * self.img_rows,
                       j * self.img_cols: (j + 1) * self.img_cols] = gen_imgs[i*n+j, :,:,0]
        plt.figure(figsize=(figsize,figsize))
        plt.imshow(figure, cmap='Greys_r')
        plt.axis('off')
        plt.savefig("/kaggle/working/pic3/GAN-G-%d.pdf" % epoch ,bbox_inches='tight')
        plt.show()

In [ ]:
#繪製d_loss, g_loss, acc
    def plot_loss(self, all_loss):
        #只繪製1000個epoch
        k = 1000
        plt.figure()
        plt.plot(all_loss[:k,0],color="blue",label="d_loss")
        plt.plot(all_loss[:k,1],color="red",linestyle="--",label="acc")
        plt.plot(all_loss[:k,2],color="green",linestyle=":",label="g_loss")
        plt.legend(loc='upper left')
        plt.axhline(y=0.5)
        plt.savefig("/kaggle/working/pic3/GAN-loss.pdf")
        plt.show()

In [ ]:
#保存模型
    def save_gan(self,path):
        self.discriminator.save(path+"/discriminator.h5")
        self.generator.save(path+"/generator.h5")
        self.combined.save(path+"/combined.h5")

In [ ]:
#載入模型
def load_gan(self,path):
    self.discriminator = load_model(path+"/discriminator.h5")
    self.generator = load_model(path+"/generator.h5")
    self.combined = load_model(path+"/combined.h5")
    

In [ ]:
#開始訓練

if __name__ == '__main__':
    gan = GAN()

In [ ]:
gan.train(epochs=30000, batch_size=28, sample_interval=3000, print_log=False)

In [ ]:
gan.sample_images(1,10)

In [ ]:
gan.sample_images(2,10)

In [ ]:
gan.sample_images(3,10)

In [ ]:
gan.sample_images(4,10)

In [ ]:
gan.sample_images(5,10)

In [ ]:
#保存模型
gan.save_gan("/kaggle/working/model")

In [ ]:
#打包output
from pathlib import Path
import zipfile
img_root = Path('/kaggle/working/pic3')
with zipfile.ZipFile('/kaggle/working/gan_output.zip', 'w') as z:
    for img_name in img_root.iterdir():
        z.write(img_name)